In [1]:
import gymnasium as gym
from gymnasium import spaces
from gymnasium.spaces import MultiDiscrete, Discrete

In [2]:
print(gym.__version__)

0.28.1


# Creating a Custom Environment



In [65]:
# make an environment
import random
import numpy as np
class SchedulerEnvironment(gym.Env):
    def __init__(self, scenario_one = True, terminate_num = 200, mean_delay_one = 6, mean_delay_two = 4, margin_of_delay = 1):
        self.scenario_one = scenario_one
        self.mean_delay_one = mean_delay_one
        self.mean_delay_two = mean_delay_two
        self.margin_of_delay = margin_of_delay


        self.queue_one_incoming_packet = 0.0
        self.queue_two_incoming_packet = 0.0
        self.queue_best_effort_incoming_packet = 0.0 

        self.incoming_packets = dict(
            {
                0: self.queue_one_incoming_packet,
                1: self.queue_two_incoming_packet,
                2: self.queue_best_effort_incoming_packet

            }
            )


        self.step_counter = 0 # i.e. timeslots
        self.terminate_num = terminate_num

        # maximum size of the queue
        self.max_queue_size = 10
        
        # can adjust size of max delay in timeslots after initial observations
        self.MAX_DELAY = 100

#         observation space is the average of the mean delays. 
        self.observation_space = spaces.Dict({
            "queues": spaces.Box(low = 0, high = self.MAX_DELAY, shape = (3,self.max_queue_size), dtype=int),
            "avg_delays": spaces.Box(low = 0, high = self.MAX_DELAY, shape = (1,3), dtype = int)
        })

        self.queue_one_delay = [] 

        self.queue_two_delay = []

        self.queue_best_effort_delay = [] 

        self.queue_delays = dict(
            {
                0: self.queue_one_delay,
                1: self.queue_two_delay,
                2: self.queue_best_effort_delay
            }
        )

        self.queue_one_removed = 0
        self.queue_two_removed = 0
        self.queue_best_effort_removed = 0

        self.removed_packets = dict(
            {
                0: self.queue_one_removed,
                1: self.queue_two_removed,
                2: self.queue_best_effort_removed
            }
        )

        self.current_state = None
        
        self.action_space = spaces.Discrete(3)
        
        self._action_to_queue = {
                0: self.queue_delays[0],
                1: self.queue_delays[1],
                2: self.queue_delays[2]
            }

    def _increment_delay(self):
        for i in range(len(self.queue_delays)):    
            for j in range(len(self.queue_delays[i])):
                if (self.queue_delays[i][j] < self.MAX_DELAY):
                    self.queue_delays[i][j] += 1

    def _can_add_packets(self, packet_arrival, action ):
        # increment before evaluating arrival. 
        self.incoming_packets[action] += packet_arrival
        packet_arrived = self.incoming_packets[action] >= 1.0
        not_max_size = len(self.queue_delays[action]) < self.max_queue_size

        if packet_arrived and not_max_size:
            self.incoming_packets[action] -= 1.0
            self.queue_delays[action].append(0)                     

    def _add_packets(self):   
        PACKET_ARRIVAL_ONE = 0.3
        PACKET_ARRIVAL_TWO = 0.25
        PACKET_ARRIVAL_BEST_EFFORT = 0.4
        
        self._can_add_packets(PACKET_ARRIVAL_ONE, 0)
        self._can_add_packets(PACKET_ARRIVAL_TWO, 1)
        self._can_add_packets(PACKET_ARRIVAL_BEST_EFFORT, 2)

        return
     
    def _modify_states(self):
        # will not calculate delay for freshly added packets. 
        self._increment_delay()
        self._add_packets()
        return 
    
    def _get_obs(self):
        
        avg_one = self._calculate_avg_delay(self.queue_delays[0])
        avg_two = self._calculate_avg_delay(self.queue_delays[1])
        avg_best = self._calculate_avg_delay(self.queue_delays[2])

        # return average delays for each queue. 
        avg_mean_all_queues = [avg_one, avg_two, avg_best]
        
        return { 
            "queues": self.queue_delays,
            "avg_delays": avg_mean_all_queues
            }
    
    def _calculate_avg_delay(self, queue):
        queue_size = len(queue)
        if (queue_size == 0):
            return 0
        
        # sum
        sum = 0
        for i in range(queue_size):
            sum += queue[i]
        
        # avg
        avg = sum / queue_size
        return avg

    def _get_info(self):

        # return the number of packets removed in each queue. 
        
        packets_removed = [self.removed_packets[0], self.removed_packets[1], self.removed_packets[2]]

        # return the queue num.
        return [packets_removed, self.step_counter + 1]
 
    def _initialise_delays(self, queue):
        # give a random delay between 0 and 5
        size_of_queue = len(queue)
        for i in range(size_of_queue):
            queue[i] = random.randint(0, 5)

    def _set_all_zero(self):
        for i in range((3)):
            self.queue_delays[i].clear()
            self.removed_packets[i] = 0

        self.step_counter = 0
        self.current_state = None
        self.queue_one_incoming_packet = 0.0
        self.queue_two_incoming_packet = 0.0
        self.queue_best_effort_incoming_packet = 0.0 
    
    def reset(self, seed=None, options=None):
        # reset everything 
        self._set_all_zero()

        # set the size of the queue. 
        self.queue_delays[0] = self.observation_space["queues"].sample().tolist()
        self.queue_delays[1] = self.observation_space["queues"].sample().tolist()
        self.queue_delays[2] = self.observation_space["queues"].sample().tolist()

        # set seed before randomising 
        random.seed(seed)

        self._initialise_delays(self.queue_delays[0])
        self._initialise_delays(self.queue_delays[1])
        self._initialise_delays(self.queue_delays[2])

        observation = self._get_obs()
        info = self._get_info()

        if self.render_mode == "human":
            self._render_frame()

        return observation, info
    
    def _reward_function(self, action):       
        reward = 0

        mean_delays = dict(
            {
                0: self.mean_delay_one,
                1: self.mean_delay_two
            }
        )

        if (action == 2):
        
            # reward taking the best effort queue when mean delays are adjusted
            # for priority queues.
            avg_first_queue = self._calculate_avg_delay(self.queue_delays[0])
            within_margin_first_queue = ( avg_first_queue <= mean_delays[0] + self.margin_of_delay)
            
            avg_second_queue = self._calculate_avg_delay(self.queue_delays[1])
            within_margin_second_queue = ( avg_second_queue <= mean_delays[1] + self.margin_of_delay)
            
            if (within_margin_first_queue and within_margin_second_queue):
                reward = 1

            # Punish for not prioritising. 
            else:
                reward = -1

        else:
            avg_delay = self._calculate_avg_delay(self.queue_delays[0])
            # packet delay > mean delay by the margin
            if (mean_delays[action] + self.margin_of_delay < avg_delay):
                # encourage minimising the delay for this queue.
                reward = 0

            # packet delay < mean delay by the margin
            elif (mean_delays[action] - self.margin_of_delay > avg_delay):
                # discourage minimising the delay too much.
                reward = -1

            # packet delay within margin, don't perform again. 
            else:
                reward = -1

        return reward 

    def _remove_packet(self, action):
        # retrieve the packet. 
        self.queue_delays[action].pop(0)
        self.removed_packets[action] += 1

    def step(self, action):
        reward = 0

        if len(self.queue_delays[action]) == 0:
            reward = -1

        else:
            reward = self._reward_function(action)

            if self.scenario_one:
                self._remove_packet(action)
    
            else: 
                if (action == self.current_state):
                    self._remove_packet(action)

        # perform queue switch given the conditions.
        if not self.scenario_one:

            if (action != self.current_state):
                self.current_state = action

                # should consider if switching is the correct approach
                # long term. Switching twice in a row is a waste of time.
                reward -= 1
                
        self._modify_states()

        info = self._get_info()

        terminated = False
        if self.step_counter + 1 == self.terminate_num:
            terminated = True
        self.step_counter += 1

        # observation made after modifying states. 
        observation = self._get_obs()

        if self.render_mode == "human":
            self._render_frame()

        return observation, reward, terminated, False, info

#     def close(self):
#         if self.window is not None:
#             pygame.display.quit()
#             pygame.quit()

In [4]:
env = SchedulerEnvironment(terminate_num = 50) 

In [5]:
print('State space Low: ', env.observation_space["queues"].low)
print('State space High: ', env.observation_space["queues"].high)

State space Low:  [[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]
State space High:  [[100 100 100 100 100 100 100 100 100 100]
 [100 100 100 100 100 100 100 100 100 100]
 [100 100 100 100 100 100 100 100 100 100]]


In [6]:
print('State space Low: ', env.observation_space["avg_delays"].low)
print('State space High: ', env.observation_space["avg_delays"].high)

State space Low:  [[0 0 0]]
State space High:  [[100 100 100]]


In [7]:
def print_info(info):
    print("----------")
    print("current step =", info[1])
    print("-----")
    print("Packets removed from first queue:", info[0][0])
    print("Packets removed from second queue:", info[0][1])
    print("Packets removed from best-effort queue:", info[0][2])
    

In [8]:
def print_observation(observation):
    print("----------")
    print("First queue:", observation["queues"][0])
    print("Second queue:", observation["queues"][1])
    print("Best effort queue:", observation["queues"][2])
    print("-----")
    print("Average delay of first queue: {:.2f}".format(observation["avg_delays"][0]))
    print("Average delay of second queue: {:.2f}".format(observation["avg_delays"][1]))
    print("Average delay of best-effort queue: {:.2f}".format(observation["avg_delays"][2]))

In [9]:
env.action_space.sample()

0

In [10]:
obs, info = env.reset()
print_observation(obs)
print_info(info)

----------
First queue: [2, 1, 3]
Second queue: [0, 3, 1]
Best effort queue: [4, 2, 1]
-----
Average delay of first queue: 2.00
Average delay of second queue: 1.33
Average delay of best-effort queue: 2.33
----------
current step = 1
-----
Packets removed from first queue: 0
Packets removed from second queue: 0
Packets removed from best-effort queue: 0


In [11]:
def random_scheduler(seed = None):
    terminated = False 
    truncated = False

    # reset seeds the randomiser
    env.reset(seed)
    score = 0

    while not terminated:  
    #     random 
        action = random.randint(0, 2)
        obs, reward, terminated, truncated, info = env.step(action)
        score += reward
        print_observation(obs)
        print_info(info)
    print(score)

In [12]:
random_scheduler(0)

----------
First queue: [4, 4, 1]
Second queue: [5, 4]
Best effort queue: [4, 3, 4]
-----
Average delay of first queue: 3.00
Average delay of second queue: 4.50
Average delay of best-effort queue: 3.67
----------
current step = 1
-----
Packets removed from first queue: 0
Packets removed from second queue: 1
Packets removed from best-effort queue: 0
----------
First queue: [5, 5, 2]
Second queue: [6, 5]
Best effort queue: [4, 5]
-----
Average delay of first queue: 4.00
Average delay of second queue: 5.50
Average delay of best-effort queue: 4.50
----------
current step = 2
-----
Packets removed from first queue: 0
Packets removed from second queue: 1
Packets removed from best-effort queue: 1
----------
First queue: [6, 3]
Second queue: [7, 6]
Best effort queue: [5, 6, 0]
-----
Average delay of first queue: 4.50
Average delay of second queue: 6.50
Average delay of best-effort queue: 3.67
----------
current step = 3
-----
Packets removed from first queue: 1
Packets removed from second queu

Best effort queue: [3, 0]
-----
Average delay of first queue: 5.67
Average delay of second queue: 3.00
Average delay of best-effort queue: 1.50
----------
current step = 43
-----
Packets removed from first queue: 12
Packets removed from second queue: 12
Packets removed from best-effort queue: 18
----------
First queue: [7, 3, 0]
Second queue: [4, 0]
Best effort queue: [4, 1]
-----
Average delay of first queue: 3.33
Average delay of second queue: 2.00
Average delay of best-effort queue: 2.50
----------
current step = 44
-----
Packets removed from first queue: 13
Packets removed from second queue: 12
Packets removed from best-effort queue: 18
----------
First queue: [8, 4, 1]
Second queue: [5, 1]
Best effort queue: [2, 0]
-----
Average delay of first queue: 4.33
Average delay of second queue: 3.00
Average delay of best-effort queue: 1.00
----------
current step = 45
-----
Packets removed from first queue: 13
Packets removed from second queue: 12
Packets removed from best-effort queue: 19

In [13]:
random_scheduler(1)

----------
First queue: [2, 5, 1, 0]
Second queue: [1, 4]
Best effort queue: [4, 4, 6]
-----
Average delay of first queue: 2.00
Average delay of second queue: 2.50
Average delay of best-effort queue: 4.67
----------
current step = 1
-----
Packets removed from first queue: 0
Packets removed from second queue: 1
Packets removed from best-effort queue: 0
----------
First queue: [6, 2, 1]
Second queue: [2, 5, 0]
Best effort queue: [5, 5, 7]
-----
Average delay of first queue: 3.00
Average delay of second queue: 2.33
Average delay of best-effort queue: 5.67
----------
current step = 2
-----
Packets removed from first queue: 1
Packets removed from second queue: 1
Packets removed from best-effort queue: 0
----------
First queue: [3, 2]
Second queue: [3, 6, 1]
Best effort queue: [6, 6, 8, 0]
-----
Average delay of first queue: 2.50
Average delay of second queue: 3.33
Average delay of best-effort queue: 5.00
----------
current step = 3
-----
Packets removed from first queue: 2
Packets removed f

Average delay of best-effort queue: 8.50
----------
current step = 40
-----
Packets removed from first queue: 15
Packets removed from second queue: 13
Packets removed from best-effort queue: 11
----------
First queue: [0]
Second queue: []
Best effort queue: [18, 16, 13, 11, 8, 6, 3, 1]
-----
Average delay of first queue: 0.00
Average delay of second queue: 0.00
Average delay of best-effort queue: 9.50
----------
current step = 41
-----
Packets removed from first queue: 15
Packets removed from second queue: 13
Packets removed from best-effort queue: 11
----------
First queue: []
Second queue: [0]
Best effort queue: [19, 17, 14, 12, 9, 7, 4, 2]
-----
Average delay of first queue: 0.00
Average delay of second queue: 0.00
Average delay of best-effort queue: 10.50
----------
current step = 42
-----
Packets removed from first queue: 16
Packets removed from second queue: 13
Packets removed from best-effort queue: 11
----------
First queue: []
Second queue: []
Best effort queue: [20, 18, 15, 1

In [14]:
random_scheduler(2)

----------
First queue: [1, 1, 1, 0]
Second queue: [3, 2, 6]
Best effort queue: [3, 3]
-----
Average delay of first queue: 0.75
Average delay of second queue: 3.67
Average delay of best-effort queue: 3.00
----------
current step = 1
-----
Packets removed from first queue: 0
Packets removed from second queue: 0
Packets removed from best-effort queue: 1
----------
First queue: [2, 2, 1]
Second queue: [4, 3, 7]
Best effort queue: [4, 4]
-----
Average delay of first queue: 1.67
Average delay of second queue: 4.67
Average delay of best-effort queue: 4.00
----------
current step = 2
-----
Packets removed from first queue: 1
Packets removed from second queue: 0
Packets removed from best-effort queue: 1
----------
First queue: [3, 3, 2]
Second queue: [5, 4, 8]
Best effort queue: [5, 0]
-----
Average delay of first queue: 2.67
Average delay of second queue: 5.67
Average delay of best-effort queue: 2.50
----------
current step = 3
-----
Packets removed from first queue: 1
Packets removed from se

Packets removed from first queue: 13
Packets removed from second queue: 9
Packets removed from best-effort queue: 13
----------
First queue: [4, 1]
Second queue: [6, 2]
Best effort queue: [10, 8, 5, 3, 0]
-----
Average delay of first queue: 2.50
Average delay of second queue: 4.00
Average delay of best-effort queue: 5.20
----------
current step = 38
-----
Packets removed from first queue: 13
Packets removed from second queue: 10
Packets removed from best-effort queue: 13
----------
First queue: [5, 2]
Second queue: [7, 3]
Best effort queue: [9, 6, 4, 1]
-----
Average delay of first queue: 3.50
Average delay of second queue: 5.00
Average delay of best-effort queue: 5.00
----------
current step = 39
-----
Packets removed from first queue: 13
Packets removed from second queue: 10
Packets removed from best-effort queue: 14
----------
First queue: [6, 3]
Second queue: [8, 4, 0]
Best effort queue: [7, 5, 2, 0]
-----
Average delay of first queue: 4.50
Average delay of second queue: 4.00
Avera

FIFO tries to minimise all queues, which means the average delays for each queue converges. 

In [15]:
class EDFScheduler():  
    
    def __init__(self, scheduler_environment):
        self.edf_env = scheduler_environment
        self.current_packet_removed_one = 0
        self.current_packet_removed_two = 0
        self.current_packet_removed_best = 0
        
#         random intialisation values picked with the same mean delays. 
        self.deadline_one = 6
        self.deadline_two = 4
        self.deadline_best_effort = 10
        
        self.deadlines = np.array([self.deadline_one, self.deadline_two, self.deadline_best_effort])
    
#     obs must be the observation returned by schedulerEnvironment. 
    def _edf_action(self, mean_delay_one, mean_delay_two, mean_delay_best_effort, info):
        
        expected_mean_delays = [mean_delay_one, mean_delay_two, mean_delay_best_effort]
        
#         random initial values. 
        mean_delay_one_increment = 6
        mean_delay_two_increment = 4
        mean_delay_best_effort_increment = 10
        
        increments = [mean_delay_one_increment, mean_delay_two_increment, mean_delay_best_effort_increment]

#         ideally we only want to change the queue that was changed.
#         how to obtain num? using get info we retrieve the previous packets + packets removed.
        old_packet_removed_one = self.current_packet_removed_one
        old_packet_removed_two = self.current_packet_removed_two
        old_packet_removed_best = self.current_packet_removed_best
        
#         info[1][num] = number of removed packets for queue num. 
        self.current_packet_removed_one = info[0][0]
        self.current_packet_removed_two = info[0][1]
        self.current_packet_removed_best = info[0][2]
        
        difference = np.array([self.current_packet_removed_one - old_packet_removed_one])
        difference = np.append(difference, self.current_packet_removed_two - old_packet_removed_two)
        difference = np.append(difference, self.current_packet_removed_best - old_packet_removed_best)
        
#         the queue which the next deadline is to be decided
        queue_deadline = np.argmax(difference, axis = 0)
    
#         increment deadlines given the conditions: mean delay < expected delay.  
        if (info[0][queue_deadline] < expected_mean_delays[queue_deadline]):
#         increment by the amount which the mean delay was off from the expected delay.
            self.deadlines[queue_deadline] += increments[queue_deadline] 
            
#         constant that modifies the level which the difference between mean delays is increased.
#         this factor is multiplication as the values mean values are small for queue 1 and 2. 
            FACTOR_DEADLINE = 3
            increment_deadline = (expected_mean_delays[queue_deadline] - info[0][queue_deadline]) * FACTOR_DEADLINE
            self.deadlines[queue_deadline] += increment_deadline
            
        else:
#             increment normally.
            self.deadlines[queue_deadline] += increments[queue_deadline]
            
#     choose the smallest deadline to return
        return np.argmin(self.deadlines, axis = 0)
    
#     mean_delay_one += mean_delay_one_increment
    def perform_scheduling(self, mean_delay_one, mean_delay_two, mean_delay_best_effort, seed = None):
        terminated = False 
        truncated = False
        obs, info = self.edf_env.reset(seed)
        score = 0
        while not terminated: 

        #     edf
            action = self._edf_action(mean_delay_one, mean_delay_two, mean_delay_best_effort, info)
            obs, reward, terminated, truncated, info = self.edf_env.step(action)
            score += reward
            print_observation(obs)
            print_info(info)
        print(score)
    

In [16]:
# predefine the expected mean delay for first queue, second queue and best effort queue. 
mean_delay_one = 6
mean_delay_two = 4
mean_delay_best_effort = 10

edf_env = SchedulerEnvironment(terminate_num = 50, mean_delay_one = mean_delay_one, mean_delay_two = mean_delay_two)


In [17]:
edf_scheduler = EDFScheduler(edf_env)

In [18]:
edf_scheduler.perform_scheduling(mean_delay_one, mean_delay_two, mean_delay_best_effort, 0)

----------
First queue: [4, 4, 1]
Second queue: [5, 4]
Best effort queue: [4, 3, 4]
-----
Average delay of first queue: 3.00
Average delay of second queue: 4.50
Average delay of best-effort queue: 3.67
----------
current step = 1
-----
Packets removed from first queue: 0
Packets removed from second queue: 1
Packets removed from best-effort queue: 0
----------
First queue: [5, 5, 2]
Second queue: [6, 5]
Best effort queue: [4, 5]
-----
Average delay of first queue: 4.00
Average delay of second queue: 5.50
Average delay of best-effort queue: 4.50
----------
current step = 2
-----
Packets removed from first queue: 0
Packets removed from second queue: 1
Packets removed from best-effort queue: 1
----------
First queue: [6, 6, 3]
Second queue: [6]
Best effort queue: [5, 6, 0]
-----
Average delay of first queue: 5.00
Average delay of second queue: 6.00
Average delay of best-effort queue: 3.67
----------
current step = 3
-----
Packets removed from first queue: 0
Packets removed from second queu

Second queue: []
Best effort queue: [37, 31, 29, 26, 24, 21, 19, 16, 14, 11]
-----
Average delay of first queue: 21.80
Average delay of second queue: 0.00
Average delay of best-effort queue: 22.80
----------
current step = 34
-----
Packets removed from first queue: 1
Packets removed from second queue: 11
Packets removed from best-effort queue: 2
----------
First queue: [38, 35, 31, 28, 24, 21, 18, 14, 11, 8]
Second queue: []
Best effort queue: [38, 32, 30, 27, 25, 22, 20, 17, 15, 12]
-----
Average delay of first queue: 22.80
Average delay of second queue: 0.00
Average delay of best-effort queue: 23.80
----------
current step = 35
-----
Packets removed from first queue: 1
Packets removed from second queue: 11
Packets removed from best-effort queue: 2
----------
First queue: [39, 36, 32, 29, 25, 22, 19, 15, 12, 9]
Second queue: [0]
Best effort queue: [39, 33, 31, 28, 26, 23, 21, 18, 16, 13]
-----
Average delay of first queue: 23.80
Average delay of second queue: 0.00
Average delay of bes

In [19]:
edf_scheduler.perform_scheduling(mean_delay_one, mean_delay_two, mean_delay_best_effort, 1)

----------
First queue: [2, 5, 1, 0]
Second queue: [3, 1, 4]
Best effort queue: [4, 6, 0]
-----
Average delay of first queue: 2.00
Average delay of second queue: 2.67
Average delay of best-effort queue: 3.33
----------
current step = 1
-----
Packets removed from first queue: 0
Packets removed from second queue: 0
Packets removed from best-effort queue: 1
----------
First queue: [3, 6, 2, 1, 0]
Second queue: [2, 5, 0]
Best effort queue: [5, 7, 1, 0]
-----
Average delay of first queue: 2.40
Average delay of second queue: 2.33
Average delay of best-effort queue: 3.25
----------
current step = 2
-----
Packets removed from first queue: 0
Packets removed from second queue: 1
Packets removed from best-effort queue: 1
----------
First queue: [4, 7, 3, 2, 1, 0]
Second queue: [6, 1]
Best effort queue: [6, 8, 2, 1, 0]
-----
Average delay of first queue: 2.83
Average delay of second queue: 3.50
Average delay of best-effort queue: 3.40
----------
current step = 3
-----
Packets removed from first qu

Average delay of second queue: 0.00
Average delay of best-effort queue: 27.00
----------
current step = 31
-----
Packets removed from first queue: 0
Packets removed from second queue: 11
Packets removed from best-effort queue: 2
----------
First queue: [33, 36, 32, 31, 30, 29, 28, 27, 26, 25]
Second queue: []
Best effort queue: [37, 31, 30, 29, 28, 27, 26, 25, 24, 23]
-----
Average delay of first queue: 29.70
Average delay of second queue: 0.00
Average delay of best-effort queue: 28.00
----------
current step = 32
-----
Packets removed from first queue: 0
Packets removed from second queue: 11
Packets removed from best-effort queue: 2
----------
First queue: [34, 37, 33, 32, 31, 30, 29, 28, 27, 26]
Second queue: []
Best effort queue: [38, 32, 31, 30, 29, 28, 27, 26, 25, 24]
-----
Average delay of first queue: 30.70
Average delay of second queue: 0.00
Average delay of best-effort queue: 29.00
----------
current step = 33
-----
Packets removed from first queue: 0
Packets removed from seco

In [20]:
edf_scheduler.perform_scheduling(mean_delay_one, mean_delay_two, mean_delay_best_effort, 2)

----------
First queue: [1, 1, 1, 0]
Second queue: [2, 6]
Best effort queue: [6, 3, 3, 0]
-----
Average delay of first queue: 0.75
Average delay of second queue: 4.00
Average delay of best-effort queue: 3.00
----------
current step = 1
-----
Packets removed from first queue: 0
Packets removed from second queue: 1
Packets removed from best-effort queue: 0
----------
First queue: [2, 2, 2, 1, 0]
Second queue: [7]
Best effort queue: [7, 4, 4, 1, 0]
-----
Average delay of first queue: 1.40
Average delay of second queue: 7.00
Average delay of best-effort queue: 3.20
----------
current step = 2
-----
Packets removed from first queue: 0
Packets removed from second queue: 2
Packets removed from best-effort queue: 0
----------
First queue: [3, 3, 3, 2, 1, 0]
Second queue: []
Best effort queue: [8, 5, 5, 2, 1, 0]
-----
Average delay of first queue: 2.00
Average delay of second queue: 0.00
Average delay of best-effort queue: 3.50
----------
current step = 3
-----
Packets removed from first queue:

-----
Packets removed from first queue: 0
Packets removed from second queue: 9
Packets removed from best-effort queue: 1
----------
First queue: [29, 29, 29, 28, 27, 26, 25, 24, 23, 22]
Second queue: []
Best effort queue: [31, 31, 28, 27, 26, 25, 24, 23, 22, 21]
-----
Average delay of first queue: 26.20
Average delay of second queue: 0.00
Average delay of best-effort queue: 25.80
----------
current step = 29
-----
Packets removed from first queue: 0
Packets removed from second queue: 10
Packets removed from best-effort queue: 1
----------
First queue: [30, 30, 30, 29, 28, 27, 26, 25, 24, 23]
Second queue: []
Best effort queue: [32, 32, 29, 28, 27, 26, 25, 24, 23, 22]
-----
Average delay of first queue: 27.20
Average delay of second queue: 0.00
Average delay of best-effort queue: 26.80
----------
current step = 30
-----
Packets removed from first queue: 0
Packets removed from second queue: 10
Packets removed from best-effort queue: 1
----------
First queue: [31, 31, 31, 30, 29, 28, 27, 

We can see that the EDF is already performing much better than the random queue. 
We infer that the EDF scheduler attempts to keep the second queue very small. 

In [21]:
class PrioritySchedulerWrapper(SchedulerEnvironment):
    def __init__(self, scenario_one = True, terminate_num = 200, mean_delay_one = 6, mean_delay_two = 4, margin_of_delay = 1):
        super().__init__(scenario_one = scenario_one,
                       terminate_num = terminate_num,
                       mean_delay_one = mean_delay_one,
                       mean_delay_two = mean_delay_two,
                       margin_of_delay = margin_of_delay)
    
    def prioritise_action(self):
#         select the queue based on what reward can be gained.
        action = 0
        mean_delays = dict(
            {
                0: self.mean_delay_one,
                1: self.mean_delay_two
            }
        )

        # choose the best action based on the current state. 
        avg_first_queue = self._calculate_avg_delay(self.queue_delays[0])
        within_margin_first_queue = ( avg_first_queue <= mean_delays[0] + self.margin_of_delay)

        avg_second_queue = self._calculate_avg_delay(self.queue_delays[1])
        within_margin_second_queue = ( avg_second_queue <= mean_delays[1] + self.margin_of_delay)

        if (within_margin_first_queue and within_margin_second_queue):
            action = 2

        else:
#             do action = 1 if the first queue is within margin
            if (within_margin_first_queue):
                action = 1
#                 action is otherwise 0.
        return action 

In [22]:
priority_scheduler_env = PrioritySchedulerWrapper(terminate_num = 50, mean_delay_one = mean_delay_one, mean_delay_two = mean_delay_two )

In [23]:
def priority_scheduler(priority_scheduler_env, seed = None):
    terminated = False 
    truncated = False

    # reset seeds the randomiser
    priority_scheduler_env.reset(seed)
    score = 0

    while not terminated:  
    #     random 
        action = priority_scheduler_env.prioritise_action()
        obs, reward, terminated, truncated, info = priority_scheduler_env.step(action)
        score += reward
        print_observation(obs)
        print_info(info)
    print(score)

In [24]:
priority_scheduler(priority_scheduler_env, 0)

----------
First queue: [4, 4, 1]
Second queue: [3, 5, 4]
Best effort queue: [3, 4]
-----
Average delay of first queue: 3.00
Average delay of second queue: 4.00
Average delay of best-effort queue: 3.50
----------
current step = 1
-----
Packets removed from first queue: 0
Packets removed from second queue: 0
Packets removed from best-effort queue: 1
----------
First queue: [5, 5, 2]
Second queue: [4, 6, 5]
Best effort queue: [5]
-----
Average delay of first queue: 4.00
Average delay of second queue: 5.00
Average delay of best-effort queue: 5.00
----------
current step = 2
-----
Packets removed from first queue: 0
Packets removed from second queue: 0
Packets removed from best-effort queue: 2
----------
First queue: [6, 6, 3]
Second queue: [5, 7, 6]
Best effort queue: [0]
-----
Average delay of first queue: 5.00
Average delay of second queue: 6.00
Average delay of best-effort queue: 0.00
----------
current step = 3
-----
Packets removed from first queue: 0
Packets removed from second queu

Second queue: [9, 5, 1]
Best effort queue: [2, 0]
-----
Average delay of first queue: 6.00
Average delay of second queue: 5.00
Average delay of best-effort queue: 1.00
----------
current step = 25
-----
Packets removed from first queue: 6
Packets removed from second queue: 6
Packets removed from best-effort queue: 11
----------
First queue: [12, 9, 5, 2]
Second queue: [10, 6, 2]
Best effort queue: [1]
-----
Average delay of first queue: 7.00
Average delay of second queue: 6.00
Average delay of best-effort queue: 1.00
----------
current step = 26
-----
Packets removed from first queue: 6
Packets removed from second queue: 6
Packets removed from best-effort queue: 12
----------
First queue: [13, 10, 6, 3, 0]
Second queue: [7, 3]
Best effort queue: [2]
-----
Average delay of first queue: 6.40
Average delay of second queue: 5.00
Average delay of best-effort queue: 2.00
----------
current step = 27
-----
Packets removed from first queue: 6
Packets removed from second queue: 7
Packets remove

Packets removed from best-effort queue: 21
----------
First queue: [12, 8, 5, 2]
Second queue: [9, 5, 1]
Best effort queue: [1]
-----
Average delay of first queue: 6.75
Average delay of second queue: 5.00
Average delay of best-effort queue: 1.00
----------
current step = 49
-----
Packets removed from first queue: 13
Packets removed from second queue: 12
Packets removed from best-effort queue: 21
----------
First queue: [13, 9, 6, 3]
Second queue: [10, 6, 2]
Best effort queue: [0]
-----
Average delay of first queue: 7.75
Average delay of second queue: 6.00
Average delay of best-effort queue: 0.00
----------
current step = 50
-----
Packets removed from first queue: 13
Packets removed from second queue: 12
Packets removed from best-effort queue: 22
16


In [25]:
priority_scheduler(priority_scheduler_env, 1)

----------
First queue: [2, 5, 1, 0]
Second queue: [3, 1, 4]
Best effort queue: [4, 6]
-----
Average delay of first queue: 2.00
Average delay of second queue: 2.67
Average delay of best-effort queue: 5.00
----------
current step = 1
-----
Packets removed from first queue: 0
Packets removed from second queue: 0
Packets removed from best-effort queue: 1
----------
First queue: [3, 6, 2, 1]
Second queue: [4, 2, 5, 0]
Best effort queue: [7]
-----
Average delay of first queue: 3.00
Average delay of second queue: 2.75
Average delay of best-effort queue: 7.00
----------
current step = 2
-----
Packets removed from first queue: 0
Packets removed from second queue: 0
Packets removed from best-effort queue: 2
----------
First queue: [4, 7, 3, 2]
Second queue: [5, 3, 6, 1]
Best effort queue: [0]
-----
Average delay of first queue: 4.00
Average delay of second queue: 3.75
Average delay of best-effort queue: 0.00
----------
current step = 3
-----
Packets removed from first queue: 0
Packets removed f

Second queue: [8, 4, 0]
Best effort queue: [1]
-----
Average delay of first queue: 7.00
Average delay of second queue: 4.00
Average delay of best-effort queue: 1.00
----------
current step = 46
-----
Packets removed from first queue: 13
Packets removed from second queue: 12
Packets removed from best-effort queue: 20
----------
First queue: [13, 10, 6, 3, 0]
Second queue: [9, 5, 1]
Best effort queue: []
-----
Average delay of first queue: 6.40
Average delay of second queue: 5.00
Average delay of best-effort queue: 0.00
----------
current step = 47
-----
Packets removed from first queue: 13
Packets removed from second queue: 12
Packets removed from best-effort queue: 21
----------
First queue: [14, 11, 7, 4, 1]
Second queue: [10, 6, 2]
Best effort queue: [0]
-----
Average delay of first queue: 7.40
Average delay of second queue: 6.00
Average delay of best-effort queue: 0.00
----------
current step = 48
-----
Packets removed from first queue: 13
Packets removed from second queue: 12
Packe

In [26]:
priority_scheduler(priority_scheduler_env, 2)

----------
First queue: [1, 1, 1, 0]
Second queue: [3, 2, 6]
Best effort queue: [3, 3]
-----
Average delay of first queue: 0.75
Average delay of second queue: 3.67
Average delay of best-effort queue: 3.00
----------
current step = 1
-----
Packets removed from first queue: 0
Packets removed from second queue: 0
Packets removed from best-effort queue: 1
----------
First queue: [2, 2, 2, 1]
Second queue: [4, 3, 7]
Best effort queue: [4]
-----
Average delay of first queue: 1.75
Average delay of second queue: 4.67
Average delay of best-effort queue: 4.00
----------
current step = 2
-----
Packets removed from first queue: 0
Packets removed from second queue: 0
Packets removed from best-effort queue: 2
----------
First queue: [3, 3, 3, 2]
Second queue: [5, 4, 8]
Best effort queue: [0]
-----
Average delay of first queue: 2.75
Average delay of second queue: 5.67
Average delay of best-effort queue: 0.00
----------
current step = 3
-----
Packets removed from first queue: 0
Packets removed from se

Average delay of first queue: 7.25
Average delay of second queue: 5.00
Average delay of best-effort queue: 1.50
----------
current step = 43
-----
Packets removed from first queue: 12
Packets removed from second queue: 11
Packets removed from best-effort queue: 18
----------
First queue: [10, 7, 3, 0]
Second queue: [8, 4, 0]
Best effort queue: [4, 1]
-----
Average delay of first queue: 5.00
Average delay of second queue: 4.00
Average delay of best-effort queue: 2.50
----------
current step = 44
-----
Packets removed from first queue: 13
Packets removed from second queue: 11
Packets removed from best-effort queue: 18
----------
First queue: [11, 8, 4, 1]
Second queue: [9, 5, 1]
Best effort queue: [2, 0]
-----
Average delay of first queue: 6.00
Average delay of second queue: 5.00
Average delay of best-effort queue: 1.00
----------
current step = 45
-----
Packets removed from first queue: 13
Packets removed from second queue: 11
Packets removed from best-effort queue: 19
----------
First 

This one has the best performance for scenario one. 

### QLearning Algorithm

In [66]:
def discretize_state(state):

    if isinstance(state, tuple):
        queues = state[0]['avg_delays']
        q1 = state[0]['avg_delays'][0]
        q2 = state[0]['avg_delays'][1]
        q3 = state[0]['avg_delays'][2]
    else:
        queues = state['avg_delays']
        q1 = state['avg_delays'][0]
        q2 = state['avg_delays'][1]
        q3 = state['avg_delays'][2]
        
#     for q in queues:
#         while len(queues[q]) < 10:
#             queues[q].append(0)

    queue_lists = [int(q1), int(q2), int(q3)]  
    arr = np.array(queue_lists)
    
    return arr

def get_state_index(state):
    state_index = state[0] * 101 ** 2 + state[1] * 101 + state[2]
    return state_index


def QLearning(env, QTable, learning, discount, epsilon, episodes):
    # Env: The OpenAI gym environment
    # Q: Initial Q table
    # learning: Learning Rate of Q learing
    # discount: discount factor (gamma)
    # epsilon: epsilon for exploration vs exploitation
    # episodes: number of episodes to run when learing the Q table
    
    # Initialize variables to hold rewards
    reward_list = []
    
    START_EPSILON_DECAYING = 1
    END_EPSILON_DECAYING = episodes // 2
    epsilon_decay_value = epsilon / (END_EPSILON_DECAYING - START_EPSILON_DECAYING)

    for episode in range(episodes):
        done = False
        total_reward, reward = 0,0
        # get the initial state
        state = env.reset()
        observation, info = env.reset()
        # TO DO: DONT NEED AS OUR SPACE IS ALREADY DISCRETE
        discretState = discretize_state(state)
        print(discretState)
        state_index = get_state_index(discretState)
        print(state_index)
      
        
        steps = 0;
        while done != True:   
                
            # Determine next action - epsilon greedy strategy for explore vs exploitation
            if np.random.random() < 1 - epsilon:
                # select the best action according to Qtable (exploitation)
                action = np.argmax(QTable[state_index]) % 3
                print(action)
            else:
                # select a random action (exploration)
                action = env.action_space.sample()
                
                
            # Step and Get the next state and reward
            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated 
            discretStateNew = discretize_state(next_state)

            # TO DO: Don't think there is a terminal state
                
            # Update the Q table
            QTable[discretState, action] = QTable[discretState, action] + \
                learning * (reward + discount * np.max(QTable[discretStateNew]) - QTable[discretState, action])
#             QTable[state, action] = QTable[state, action] + \
#                 learning * (reward + discount * np.max(QTable[next_state[0], next_state[1]]) - QTable[state, action])

                        
            # Update variables
            total_reward += reward
            discretState = discretStateNew 
#             state = new_state
            steps = steps + 1
            
        # Update epsilon
        if END_EPSILON_DECAYING >= episode and episode >= START_EPSILON_DECAYING:
            epsilon -= epsilon_decay_value
            
        # Track rewards
        reward_list.append(total_reward)
        
        if (episode + 1) % 100 == 0:
            ave_reward = np.mean(reward_list)
            reward_list = []
            
        if ( episode +1) % 100 == 0:    
            print('Episode {} Average Reward: {}'.format(episode+1, ave_reward))


    env.close()
    
    return QTable


In [67]:
env = SchedulerEnvironment(terminate_num = 50) 
num_states = 101 ** 3

print(num_states)
Q = np.random.uniform(low = -1, high = 1, size = (num_states, 3, 3))
# Run Q-learning algorithm
Q = QLearning(env, Q, 0.4, 0.8, 0.7, 100)

1030301
[3 1 1]
30705
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
[1 2 2]
10405
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
[1 0 2]
10203
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
[2 3 4]
20709
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
[2 3 2]
20707
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
[2 3 1]
20706
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
[3 3 2]
30908
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
[2 3 2]
20707
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
[2 2 2]
20606
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
[1 2 4]
10407
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
[3 3 2]
30908
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
[4 2 2]
41008
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
[2 2 1]
20605
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
[3 3 1]
30907
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
[2 2 1]
20605
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
[2 1 2]
20505
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
[1 2 2]
10405
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
[0 3 3]
306
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
[2

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
[1 1 1]
10303
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
[2 2 3]
20607
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
[2 3 1]
20706
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
[4 2 2]
41008
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
[2 2 2]
20606
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
[4 3 2]
41109
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
[2 2 1]
20605
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
Episode 100 Average Reward: -38.36


In [ ]:
state = env.reset()
observation, info = env.reset()
state_adj = discretize_state(state[0], env.observation_space["avg_delays"].low)
done = False
step_index = 0
while done != True:
    action = np.argmax(Q[state_adj[0], state_adj[1]]) 
    state, reward, terminated, truncated, info = env.step(action)
    done = terminated or truncated 
    state_adj = discretize_state(state, env.observation_space.low)
#     show_state(env, step=step_index, info='State ({},{}) Reward: {}'.format(d_state[0], d_state[1], reward))
    step_index = step_index + 1